In [1]:

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization,Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
import kagglehub
from tensorflow.keras.preprocessing.image import ImageDataGenerator

c:\Users\Hossam Mostafa\NTI-Repo-clone\NTI_Training\Flower\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

import kagglehub

# Download latest version
path = kagglehub.dataset_download("alxmamaev/flowers-recognition")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\Hossam Mostafa\.cache\kagglehub\datasets\alxmamaev\flowers-recognition\versions\2


In [3]:

datagen = ImageDataGenerator(rescale=1./255,
                             rotation_range = 30,
                             width_shift_range = .1,
                             height_shift_range = .1,
                             shear_range = .1,
                             zoom_range = .3,
                             horizontal_flip = True,
                             validation_split = .2,
                             fill_mode = 'nearest')

In [8]:
# The 'path' variable holds the correct local directory
# We also need to point to the 'flowers' subfolder inside it
import os
flowers_directory = os.path.join(path, "flowers")

train_generator = datagen.flow_from_directory(
    flowers_directory,             # Use the correct local path
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',      # Changed from 'binary' to 'categorical'
    subset='training',
    shuffle=True
)

Found 3457 images belonging to 5 classes.


In [12]:
# The 'path' variable holds the correct local directory
# We also need to point to the 'flowers' subfolder inside it
flowers_directory = os.path.join(path, "flowers")

# Use the 'val_datagen' which only rescales the images
validation_generator = datagen.flow_from_directory(
    flowers_directory,        # Use the correct local path
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical', # Must be 'categorical' for 5 classes
    subset='validation',
    shuffle=False
)

Found 860 images belonging to 5 classes.


## Model


In [ ]:
model = Sequential([Input(shape=(128, 128, 3)),
                    Conv2D(32, (3, 3), activation='relu', padding='same'),
                    MaxPooling2D(2,2),
                    Conv2D(32, (3, 3), activation='relu', padding='same'),
                    MaxPooling2D(2, 2),
                    Flatten(),
                    Dense(16, activation='relu'),
                    Dense(8, activation='relu'),
                    Dense(5, activation='softmax')])

In [14]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [15]:

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 128, 128, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 64, 64, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 64, 64, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 32768)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 16)             │       524,304 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 100)            │           900 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 535,484 (2.04 MB)

 Trainable params: 535,484 (2.04 MB)

 Non-trainable params: 0 (0.00 B)

In [17]:


history = model.fit(train_generator, epochs=10, validation_data=validation_generator)

c:\Users\Hossam Mostafa\NTI-Repo-clone\NTI_Training\Flower\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10


ValueError: Argument `output` must have rank (ndim) `target.ndim - 1`. Received: target.shape=(None, 5), output.shape=(None, 100)

In [17]:
model.save('my_model.keras')
